# Logistic Regression Model

和前面提到的 Linear Regression 一样， Logistic Regression 也属于 Generalized Linear Model。Logistic Regression 是 Linear Regression 很直接的扩展，Logistic Regression 把 Linear Regression 的结果送入到 sigmoid 函数中，计算得到结果。

## sigmoid 函数

$\begin{align*}
sigmoid(x) = \frac{1}{1 + e^{-x}} = \frac{e^x}{e^x + 1}
\end{align*}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

x = np.linspace(-10,10,200)
y = 1 / (1 + np.exp(-1 * x))
plt.plot(x, y, 'b', label='sigmoid(x) = e^x / (e^x + 1)')
plt.xlabel('X')
plt.ylabel('sigmoid(x)')
plt.title('sigmoid(x) = e^x / (e^x + 1)')
plt.show()
plt.close()

很多人常讲，Logistic Regression 最后得到的结果是一个概率值。这个值真的是概率值么？指数分布簇可以给我们答案。

## Exponential Family

单一变量的 exponential family 是 $f(x|\theta)=h(x)e^{\eta(\theta)T(x)-A(\theta)}$

其中 $\eta(\theta)$ 是自然参数，在 Bernoulli 分布中，只有一个自然参数，那就是 $p$。另外的，$A(\theta)$ 可以表示成 $A(\theta) = f(\eta(\theta))$ 的形式。

如果 $x \sim Bernoulli(x|p) = p^x(1-p)^{1-x}$，那么就有：

$
\begin{align*}
Bernoulli(x|p) & = p^x(1-p)^{1-x} \\
& = e^{log(p^x(1-p)^{1-x})} \\
& = e^{xlog(p) + (1-x)log(1-p)} \\ 
& = e^{xlog(\frac{p}{1-p}) + log(1-p)}
\end{align*}
$

对应指数分布簇，即有 $h(x)=1$，$\eta(\theta)=log(\frac{p}{1-p})$，$T(x)=x$，$A(\theta)=-log(1-p)$。稍微对 p 做一些分析：

$
\begin{align*}
\eta(\theta) & = log(\frac{p}{1-p}) \\
\Rightarrow e^{\eta(\theta)} \cdot (1-p) & = p \\
e^{\eta(\theta)} & = p(1 + e^{\eta(\theta)}) \\
\frac{e^{\eta(\theta)}}{1+e^{\eta(\theta)}} & = p 
\end{align*}
$

所以，如果使用 $\eta(\theta)) = \mathbf{\theta}^T \mathbf{x}$，那么 $p = \frac{1}{1+e^{-\theta^T x}}$，所以有人会说 Logistic Regression 的输出是一个概率值。在传统的 Statistical Machine Learning 中，通过预先选择一个模型，然后根据数据，学习出模型的参数值。过去的一段时间内，我一度认为 Logistic Regression 的输出值，不能代表概率值，其实是部分正确的，如果说 $\eta(\theta)=\mathbf{\theta}^T \mathbf{x}$ 能真实的反映 $\eta(\theta)$ 函数，那么 $p$ 的值就是概率值；否则就不行。

类似于 Linear Regression，要通过训练数据获取 $\theta$ 的值，也需要类似的两步，设置 Loss 和使用 Learning Method。在 Logistic Regression 中，常用的损失函数称为 logloss 或者 cross-entropy，两者分别是 Statistics 和 Information Theory 视角的表述。

## logloss

简单的做一个推导

$
\begin{align*}
{logloss}_i = y_i \cdot log(\hat{y_i}) + (1 - y_i) \cdot log(1 - \hat{y_i})
\end{align*}
$

这样的话，如果 $y=0$，那么只有 $(1 - y) \cdot log(1 - \hat{y})$ 生效，且 $\hat{y}$ 要趋近于 0 才能使得 loss 最小；如果 $y=1$，那么只有 $y \cdot log(\hat{y})$ 生效，且 $\hat{y}$ 要趋近于 1 才能是 loss 最小。

$
\begin{align*}
logloss & = \frac{1}{N} \sum\limits_i^N [y_i \cdot log(\hat{y_i}) + (1 - y_i) \cdot log(1 - \hat{y_i})] \\
& \Rightarrow \frac{1}{N} \sum\limits_i^N  [y_i \cdot log (\frac{e^{\theta^T \mathbf{x_i}}}{1 + e^{\theta^T \mathbf{x_i}}}) + log (\frac{1}{1 + e^{\theta^T \mathbf{x_i}}}) - y \cdot log (\frac{1}{1 + e^{\theta^T \mathbf{x_i}}})] \\
& = \frac{1}{N} \sum\limits_i^N [y_i \cdot (\theta^T \mathbf{x_i}) - y_i \cdot log(1 + e^{\theta^T \mathbf{x_i}}) - log(1 + e^{\theta^T \mathbf{x_i}}) + y_i \cdot log(1 + e^{\theta^T \mathbf{x_i}})]\\
& = \frac{1}{N} \sum\limits_i^N [y_i \cdot (\theta^T \mathbf{x_i}) - log(1 + e^{\theta^T \mathbf{x_i}})]
\end{align*}
$


对于某一个参数 $\theta_i$，计算 gradient：

$
\begin{align*}
\frac{\partial loss}{\theta_i} & = \frac{1}{N} \sum\limits_i^N [y_i \cdot x_i - \frac{e^{\theta^T \mathbf{x_i}}}{1 + e^{\theta^T \mathbf{x_i}}} x_i] \\
& = \frac{1}{N} \sum\limits_i^N (y_i \cdot x_i - \hat{y_i} x_i) = \frac{1}{N} \sum\limits_i^N (y_i - \hat{y_i}) x_i
\end{align*}
$

使用 iris 数据集来进行 Logistic Regression 的示例。

In [16]:
# numpy
import numpy as np
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)

index = np.where(y < 2)

X = X[index]
y = y[index]

X = np.c_[X, np.ones(len(index[0]))]

In [80]:
# numpy
np_theta = np.random.randn(X.shape[1], 1)
LEARNING_RATE = 1e-5

BATCH_SIZE = 16
EPOCH = 30000
PRINT_STEP = EPOCH / 10

for epoch in range(EPOCH):
    index = np.random.randint(0, X.shape[0], size=BATCH_SIZE)
    sample_x = X[index].reshape(X.shape[1], BATCH_SIZE)
    sample_y = y[index]

    y_pred = 1 / (1 + np.exp(-1 * np.dot(np_theta.T, sample_x)))
    logloss = np.sum(np.multiply(sample_y, np.log(y_pred)) + np.multiply((1 - sample_y), np.log(1 - y_pred))) / BATCH_SIZE

    o = np.sum(np.multiply(sample_y - y_pred, sample_x), 1).reshape(X.shape[1], 1)
    np_theta -= LEARNING_RATE * o / BATCH_SIZE

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, logloss))

print(np_theta)

EPOCH: 0, loss: -2.370534
EPOCH: 3000, loss: -1.964829
EPOCH: 6000, loss: -2.511892
EPOCH: 9000, loss: -2.162495
EPOCH: 12000, loss: -2.855219
EPOCH: 15000, loss: -2.525736
EPOCH: 18000, loss: -3.211407
EPOCH: 21000, loss: -3.219594
EPOCH: 24000, loss: -2.564125
EPOCH: 27000, loss: -3.675666
[[-0.00920321]
 [ 0.51700381]
 [ 0.2214591 ]
 [ 0.84188067]
 [ 1.37935963]]


In [75]:
# PyTorch

import torch

device = torch.device('cpu')
dtype = torch.double

INPUT_DIMENSION, OUTPUT_DIMENSION = X.shape[1], 1

theta = torch.randn(INPUT_DIMENSION, OUTPUT_DIMENSION, device=device, dtype=dtype, requires_grad=True)
LEARNING_RATE = 1e-5

BATCH_SIZE = 16
EPOCH = 3000
PRINT_STEP = EPOCH / 10

for epoch in range(EPOCH):
    index = np.random.randint(0, X.shape[0], size=BATCH_SIZE)
    sample_x = torch.from_numpy(X[index]).reshape(INPUT_DIMENSION, BATCH_SIZE)
    sample_y = torch.from_numpy(y[index])

    y_pred = 1 / (1 + torch.exp(-1 * theta.T.mm(sample_x)))
    logloss = torch.sum(torch.mul(sample_y, torch.log(y_pred)) + torch.mul((1 - sample_y), torch.log(1 - y_pred))) / BATCH_SIZE

    logloss.backward()

    with torch.no_grad():
        theta -= LEARNING_RATE * theta.grad

        # Manually zero the gradients after updating weights
        theta.grad.zero_()

    if epoch % PRINT_STEP == 0:
        print('EPOCH: %d, loss: %f' % (epoch, logloss))

print(theta)

EPOCH: 0, loss: -6.026857
EPOCH: 300, loss: -7.379591
EPOCH: 600, loss: -7.419720
EPOCH: 900, loss: -8.910971
EPOCH: 1200, loss: -5.656529
EPOCH: 1500, loss: -6.962538
EPOCH: 1800, loss: -6.705369
EPOCH: 2100, loss: -6.862127
EPOCH: 2400, loss: -6.425948
EPOCH: 2700, loss: -8.399547
tensor([[0.3517],
        [1.3085],
        [0.3771],
        [2.3578],
        [1.0618]], dtype=torch.float64, requires_grad=True)


## AUC

评价一个二元分类器的好坏，可以使用 AUC，AUC 的值越接近于 1 越好。

In [83]:
# numpy score

np_score = 1 / (1 + np.exp(-1 * np.dot(np_theta.T, X.T)))
py_theta = np.array([[0.3517],
                     [1.3085],
                     [0.3771],
                     [2.3578],
                     [1.0618]])
py_score = 1 / (1 + np.exp(-1 * np.dot(py_theta.T, X.T)))

from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y, np_score.T)
print('numpy version auc is %f' % metrics.auc(fpr,tpr))

fpr, tpr, thresholds = metrics.roc_curve(y, py_score.T)
print('PyTorch version auc is %f' % metrics.auc(fpr,tpr))

numpy version auc is 0.996400
PyTorch version auc is 0.991600
